# Introduction
This kernel has been created by the [Information Systems Lab](http://islab.uom.gr) to introduce students of the [University of Macedonia](http://www.uom.gr/index.php?tmima=2&categorymenu=2), Greece to Machine Learning & Data Science.

## The Instacart competition
Instacart is an American company that operates as a same-day grocery delivery service. Customers select groceries through a web application from various retailers and delivered by a personal shopper. Instacart's service is mainly provided through a smartphone app, available on iOS and Android platforms, apart from its website.

In 2017 Instacart organised a Kaggle competition and provided to the community a sample dataset of over 3 million grocery orders from more than 200,000 Instacart users. The orders include 32 million basket items and 50,000 unique products. The objective of the competition was participants to **predict which previously purchased products will be in a user’s next order**.

## Objective
The objective of this Kernel is to introduce students to predictive business analytics with Python through the Instacart case. 

By the time you finish this example, you will be able to:
* Describe the steps of creating a predictive analytics model
* Use Python and Pandas package to manipulate data
* Use Python and Pandas package to create, combine, and delete DataFrames
* Use Logistic Regression to create a predictive model
* Apply the predictive model in order to make a prediction
* Create a submission file for the competition of Instacart

## Problem definition
The data that Instacart opened up include orders of 200,000 Instacart users with each user having between 4 and 100 orders. Instacart indicates each order in the data as prior, train or test. Prior orders describe the **past behaviour** of a user while train and test orders regard the **future behaviour that we need to predict**. 

As a result, we want to predict which previously purchased products (prior orders) will be in a user’s next order (train and test orders). 

For the train orders Instacart reveals the results (i.e., the ordered products) while for the test orders we do not have this piece of information. Moreover, the future order of each user can be either train or test meaning that each user will be either a train or a test user. 

The setting of the Instacart problem is described in the figure below (orders with yellow color denotes future orders of a user). 

<img src="https://i.imgur.com/S0Miw3m.png" width="350">

Each user has purchased various products during their prior orders. Moreover, for each user we know the order_id of their future order. The goal is to predict which of these products will be in a user's future order. 

This is a **classification problem** because we need to predict whether each pair of user and product is a reorder or not. This is indicated by the value of the reordered variable, i.e. reordered=1 or reordered=0 (see figure below). 

<img src="https://i.imgur.com/SxK2gsR.png" width="350">

As a result we need to come up and calculate various **predictor variables (X)** that will describe the characteristics of a product and the behaviour of a user regarding one or multiple products. We will do so by analysing the prior orders of the dataset. We will then use the train users to create a predictive model and the test users to make our actual prediction. As a result we create a table as the following one and we train an algorithm based on predictor variables (X) and response variable (Y).

<img src="https://i.imgur.com/Yb1CKAF.png" width="600">

## Method
Our method includes the following steps:
1. <b>Import and reshape data</b>: This step includes loading CSV files into pandas DataFrames, tranform character variables to categorical variables, and create a supportive table.
2. <b>Create predictor variables</b>: This step includes identifying and calculating predictor variables (aka features) from the initial datasets provided by Instacart. 
3. <b>Create train and test DataFrames</b>: In this step we create two distinct pandas DataFrames that will be used in the creation and the use of the predictive model.
4. <b>Create predictive model (fit)</b>: In this step we employ Logistic Regression to create the predictive model through the train dataset.
5. <b>Apply predictive model (predict)</b>: This step includes applying the model to predict the 'reordered' variable for the test dataset.
6. <b>Create submission file</b>: In this final step we create the submission file with our predictions for Instacart's competition.
7. <b>Get F1 score</b>: In this step we submit the produced and file and get the F1 score describing the accuracy of our prediction model.

# 1. Import and Reshape Data 
First we load the necessary Python packages and then we import the CSV files that were provided by Instacart.

## 1.1 Import the required packages
The garbage collector (package gc), attempts to reclaim garbage, or memory occupied by objects (e.g., DataFrames) that are no longer in use by Python ([ref1](https://www.techopedia.com/definition/1083/garbage-collection-gc-general-programming), [ref2](https://en.wikipedia.org/wiki/Garbage_collection_(computer_science)). This package will eliminate our risk to exceed the 16GB threshold of available RAM that Kaggle offers.

The **"as"** reserved word is to define an alias to the package. The alias help us to call easier a package in our code.

In [1]:
# For data manipulation
import pandas as pd 


# Garbage Collector to free up memory
import gc                         
gc.enable()                       # Activate 

## 1.2 Load data from the CSV files
Instacart provides 6 CSV files, which we have to load into Python. Towards this end, we use the .read_csv() function, which is included in the Pandas package. Reading in data with the .read_csv( ) function returns a DataFrame.

In [5]:
orders = pd.read_csv('C:/Users/User/Desktop/instacart-market-basket-analysisorders.csv')
order_products_train = pd.read_csv('C:/Users/User/Desktop/instacart-market-basket-analysis/order_products__train.csv')
order_products_prior = pd.read_csv('C:/Users/User/Desktop/instacart-market-basket-analysis/order_products__prior.csv')
products = pd.read_csv('C:/Users/User/Desktop/instacart-market-basket-analysis/products.csv')
aisles = pd.read_csv('C:/Users/User/Desktop/instacart-market-basket-analysis/aisles.csv')
departments = pd.read_csv('C:/Users/User/Desktop/instacart-market-basket-analysis/departments.csv')

FileNotFoundError: [Errno 2] File C:/Users/User/Desktop/instacart-market-basket-analysisorders.csv does not exist: 'C:/Users/User/Desktop/instacart-market-basket-analysisorders.csv'

This step results in the following DataFrames:
* <b>orders</b>: This table includes all orders, namely prior, train, and test. It has single primary key (<b>order_id</b>).
* <b>order_products_train</b>: This table includes training orders. It has a composite primary key (<b>order_id and product_id</b>) and indicates whether a product in an order is a reorder or not (through the reordered variable).
* <b>order_products_prior </b>: This table includes prior orders. It has a composite primary key (<b>order_id and product_id</b>) and indicates whether a product in an order is a reorder or not (through the reordered variable).
* <b>products</b>: This table includes all products. It has a single primary key (<b>product_id</b>)
* <b>aisles</b>: This table includes all aisles. It has a single primary key (<b>aisle_id</b>)
* <b>departments</b>: This table includes all departments. It has a single primary key (<b>department_id</b>)

If you want to reduce the execution time of this Kernel you can use the following piece of code by uncomment it. This will trim the orders DataFrame and will keep a 10% random sample of the users. You can use this for experimentation.

In [3]:

#### Remove triple quotes to trim your dataset and experiment with your data
# COMMANDS FOR CODING TESTING - Get 10% of users 
orders = orders.loc[orders.user_id.isin(orders.user_id.drop_duplicates().sample(frac=0.1, random_state=25))] 


We now use the .head( ) method in order to visualise the first 10 rows of these tables. Click the Output button below to see the tables.

In [4]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
54,2565571,7,prior,1,3,9,NaN
55,2402008,7,prior,2,1,19,30.0
56,121053,7,prior,3,0,18,30.0
57,1695742,7,prior,4,2,10,9.0
58,3321109,7,prior,5,5,18,3.0


In [5]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [6]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [7]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [9]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


## 1.3 Reshape data
We transform the data in order to facilitate their further analysis. First, we convert character variables into categories so we can use them in the creation of the model. In Python, a categorical variable is called category and has a fixed number of different values.

In [10]:
# We convert character variables into category. 
# In Python, a categorical variable is called category and has a fixed number of different values
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

## 1.4 Create a DataFrame with the orders and the products that have been purchased on prior orders (op)
We create a new DataFrame, named <b>op</b> which combines (merges) the DataFrames <b>orders</b> and <b>order_products_prior</b>. Bear in mind that <b>order_products_prior</b> DataFrame includes only prior orders, so the new DataFrame <b>op</b>  will contain only these observations as well. Towards this end, we use pandas' merge function with how='inner' argument, which returns records that have matching values in both DataFrames. <img src="https://i.imgur.com/zEK7FpY.jpg" width="400">

In [11]:
#Merge the orders DF with order_products_prior by their order_id, keep only these rows with order_id that they are appear on both DFs
op = orders.merge(order_products_prior, on='order_id', how='inner')
op.head()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2565571,7,prior,1,3,9,NaN,45628,1,0
1,2565571,7,prior,1,3,9,NaN,39275,2,0
2,2565571,7,prior,1,3,9,NaN,6361,3,0
3,2565571,7,prior,1,3,9,NaN,45066,4,0
4,2565571,7,prior,1,3,9,NaN,13249,5,0


The table contains for all the customers **(user_id)**: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ➡︎ the orders **(order_id)** that they have placed accompanied with: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ➡︎ the products **(product_id)** that have been bought in each order

In [12]:
op['order_number_back'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1
op.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2565571,7,prior,1,3,9,NaN,45628,1,0,20
1,2565571,7,prior,1,3,9,NaN,39275,2,0,20
2,2565571,7,prior,1,3,9,NaN,6361,3,0,20
3,2565571,7,prior,1,3,9,NaN,45066,4,0,20
4,2565571,7,prior,1,3,9,NaN,13249,5,0,20
5,2565571,7,prior,1,3,9,NaN,31683,6,0,20
6,2565571,7,prior,1,3,9,NaN,519,7,0,20
7,2565571,7,prior,1,3,9,NaN,37999,8,0,20
8,2565571,7,prior,1,3,9,NaN,49508,9,0,20
9,2565571,7,prior,1,3,9,NaN,29871,10,0,20


In [13]:
op5 = op[op.order_number_back <= 5]
op5.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
148,2425840,7,prior,16,0,13,11.0,37602,1,1,5
149,2425840,7,prior,16,0,13,11.0,40852,2,1,5
150,2425840,7,prior,16,0,13,11.0,21137,3,1,5
151,2425840,7,prior,16,0,13,11.0,17638,4,1,5
152,2425840,7,prior,16,0,13,11.0,42265,5,0,5


# 2. Create Predictor Variables
We are now ready to identify and calculate predictor variables based on the provided data. We can create various types of predictors such as:
* <b>User predictors</b> describing the behavior of a user e.g. total number of orders of a user.
* <b>Product predictors</b> describing characteristics of a product e.g. total number of times a product has been purchased.
* <b>User & product predictors</b> describing the behavior of a user towards a specific product e.g. total times a user ordered a specific product.

## 2.1 Create user predictors


### 2.1.1 Number of orders per customer 📚📝
We calculate the total number of placed orders per customer. We create a **user** DataFrame to store the results.

In [14]:
user = op.groupby('user_id')['order_number'].max().to_frame('user_t_orders') #
user = user.reset_index()
user.head()

,user_id,user_t_orders
0,7,20
1,14,13
2,22,15
3,24,18
4,29,18


In [15]:
u_reorder = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio') #
u_reorder = u_reorder.reset_index()
u_reorder.head()

,user_id,u_reordered_ratio
0,7,0.669903
1,14,0.323810
2,22,0.507246
3,24,0.526316
4,29,0.702222


In [16]:
user=user.merge(u_reorder,on='user_id',how='left')#
del u_reorder
gc.collect()

user.head()

,user_id,user_t_orders,u_reordered_ratio
0,7,20,0.669903
1,14,13,0.323810
2,22,15,0.507246
3,24,18,0.526316
4,29,18,0.702222


In [17]:
days = op.groupby('user_id')['days_since_prior_order'].mean().to_frame('mean_days')
days = days.reset_index()
days.head()

,user_id,mean_days
0,7,13.546392
1,14,24.570732
2,22,11.952381
3,24,15.162162
4,29,11.793427


In [18]:
user = user.merge(days, on='user_id', how='left')
del days
user.head()

,user_id,user_t_orders,u_reordered_ratio,mean_days
0,7,20,0.669903,13.546392
1,14,13,0.323810,24.570732
2,22,15,0.507246,11.952381
3,24,18,0.526316,15.162162
4,29,18,0.702222,11.793427


In [19]:
user_max = op.groupby(['user_id','order_id'])['add_to_cart_order'].max().to_frame('max_basket')
user_max.head()

max_basket
user_id order_id            
7       23391             11
        121053            24
        369015             5
        565870             9
        776265             8

In [20]:
user_max_ratio = user_max.groupby('user_id')['max_basket'].mean().to_frame('mean_basket')
user_max_ratio = user_max_ratio.reset_index()
user_max_ratio.head()

,user_id,mean_basket
0,7,10.300000
1,14,16.153846
2,22,4.600000
3,24,2.111111
4,29,12.500000


In [21]:
user = user.merge(user_max_ratio, on='user_id', how='left')
del user_max_ratio
gc.collect()
user.head()

,user_id,user_t_orders,u_reordered_ratio,mean_days,mean_basket
0,7,20,0.669903,13.546392,10.300000
1,14,13,0.323810,24.570732,16.153846
2,22,15,0.507246,11.952381,4.600000
3,24,18,0.526316,15.162162,2.111111
4,29,18,0.702222,11.793427,12.500000


In [22]:
del  user_max
gc.collect()
user.head()

,user_id,user_t_orders,u_reordered_ratio,mean_days,mean_basket
0,7,20,0.669903,13.546392,10.300000
1,14,13,0.323810,24.570732,16.153846
2,22,15,0.507246,11.952381,4.600000
3,24,18,0.526316,15.162162,2.111111
4,29,18,0.702222,11.793427,12.500000


# **Products**

In [23]:
# Create distinct groups for each product, count the orders, save the result for each product to a new DataFrame  
prd = op.groupby('product_id')['order_id'].count().to_frame('prd_t_purchases') #


In [24]:
# Reset the index of the DF so to bring product_id rom index to column (pre-requisite for step 2.4)
prd = prd.reset_index()
prd.head()

,product_id,prd_t_purchases
0,1,205
1,2,13
2,3,15
3,4,37
4,6,1


In [25]:
#REMOVE PRODUCTS WITH LESS THAN 40 PURCHASES
# execution time: 25 sec
# the x on lambda function is a temporary variable which represents each group
# shape[0] on a DataFrame returns the number of rows
p_reorder = op.groupby('product_id').filter(lambda x: x.shape[0] >40)#####
p_reorder.head()####

###Auto einai to diko mou filtro

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
1,2565571,7,prior,1,3,9,NaN,39275,2,0,20
2,2565571,7,prior,1,3,9,NaN,6361,3,0,20
3,2565571,7,prior,1,3,9,NaN,45066,4,0,20
4,2565571,7,prior,1,3,9,NaN,13249,5,0,20
5,2565571,7,prior,1,3,9,NaN,31683,6,0,20


In [26]:
p_reorder = op.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorder = p_reorder.reset_index()
p_reorder.head()

,product_id,p_reorder_ratio
0,1,0.692683
1,2,0.230769
2,3,0.466667
3,4,0.540541
4,6,0.000000


In [27]:
#Merge the prd DataFrame with reorder
prd = prd.merge(p_reorder, on='product_id', how='left')

#delete the reorder DataFrame
del p_reorder
gc.collect()

prd.head()

,product_id,prd_t_purchases,p_reorder_ratio
0,1,205,0.692683
1,2,13,0.230769
2,3,15,0.466667
3,4,37,0.540541
4,6,1,0.000000


In [28]:
prd['p_reorder_ratio'] = prd['p_reorder_ratio'].fillna(0) #
prd.head()

,product_id,prd_t_purchases,p_reorder_ratio
0,1,205,0.692683
1,2,13,0.230769
2,3,15,0.466667
3,4,37,0.540541
4,6,1,0.000000


In [29]:
aop = op.groupby('product_id')['add_to_cart_order'].mean().to_frame("aop_mean")
aop = aop.reset_index()
aop.head()

,product_id,aop_mean
0,1,7.878049
1,2,11.384615
2,3,4.533333
3,4,6.972973
4,6,26.000000


In [30]:
prd = prd.merge(aop, on='product_id', how='left')
del aop
gc.collect()
prd.head()

,product_id,prd_t_purchases,p_reorder_ratio,aop_mean
0,1,205,0.692683,7.878049
1,2,13,0.230769,11.384615
2,3,15,0.466667,4.533333
3,4,37,0.540541,6.972973
4,6,1,0.000000,26.000000


## 2.3 Create user-product predictors


### 2.3.1 How many times a user bought a product 📚📝
We create different groups that contain all the rows for each combination of user and product. Then with the aggregation function .count( ) we get how many times each user bought a product. We save the results on new **uxp** DataFrame.

In [31]:
# Create distinct groups for each combination of user and product, count orders, save the result for each user X product to a new DataFrame 
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_t_bought') #
uxp.head()

uxp_t_bought
user_id product_id              
7       274                    1
        519                    2
        4920                   7
        4945                   3
        6361                   5

In [32]:
# Reset the index of the DF so to bring user_id & product_id rom indices to columns (pre-requisite for step 2.4)
uxp = uxp.reset_index()
uxp.head()

,user_id,product_id,uxp_t_bought
0,7,274,1
1,7,519,2
2,7,4920,7
3,7,4945,3
4,7,6361,5


In [33]:
times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times.head()

Times_Bought_N
user_id product_id                
7       274                      1
        519                      2
        4920                     7
        4945                     3
        6361                     5

In [34]:
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders') #
total_orders.head()

,total_orders
user_id,
7,20
14,13
22,15
24,18
29,18


In [35]:
first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

,user_id,product_id,first_order_number
0,7,274,2
1,7,519,1
2,7,4920,2
3,7,4945,16
4,7,6361,1


In [36]:
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_number
0,7,20,274,2
1,7,20,519,1
2,7,20,4920,2
3,7,20,4945,16
4,7,20,6361,1


In [37]:
# The +1 includes in the difference the first order were the product has been purchased
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
span.head()

,user_id,total_orders,product_id,first_order_number,Order_Range_D
0,7,20,274,2,19
1,7,20,519,1,20
2,7,20,4920,2,19
3,7,20,4945,16,5
4,7,20,6361,1,20


In [38]:
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D
0,7,274,1,20,2,19
1,7,519,2,20,1,20
2,7,4920,7,20,2,19
3,7,4945,3,20,16,5
4,7,6361,5,20,1,20


In [39]:
#Remove temporary DataFrames
del [times, first_order_no, span]
gc.collect()

In [40]:
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D ##
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D,uxp_reorder_ratio
0,7,274,1,20,2,19,0.052632
1,7,519,2,20,1,20,0.100000
2,7,4920,7,20,2,19,0.368421
3,7,4945,3,20,16,5,0.600000
4,7,6361,5,20,1,20,0.250000


In [41]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')

del uxp_ratio
uxp.head()

,user_id,product_id,uxp_t_bought,Times_Bought_N,total_orders,first_order_number,Order_Range_D,uxp_reorder_ratio
0,7,274,1,1,20,2,19,0.052632
1,7,519,2,2,20,1,20,0.100000
2,7,4920,7,7,20,2,19,0.368421
3,7,4945,3,3,20,16,5,0.600000
4,7,6361,5,5,20,1,20,0.250000


In [42]:
uxp = uxp.drop(['total_orders', 'first_order_number', 'Order_Range_D', 'Times_Bought_N'], axis=1)
uxp.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio
0,7,274,1,0.052632
1,7,519,2,0.100000
2,7,4920,7,0.368421
3,7,4945,3,0.600000
4,7,6361,5,0.250000


In [43]:
last_five = op5.groupby(['user_id','product_id'])['order_id'].count().to_frame('times_last5')
last_five.head(10)

times_last5
user_id product_id             
7       4920                  2
        4945                  3
        8277                  1
        8518                  1
        9598                  1
        10504                 1
        11520                 1
        12196                 2
        13198                 2
        15592                 2

In [44]:
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')

del last_five
uxp.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5
0,7,274,1,0.052632,NaN
1,7,519,2,0.100000,NaN
2,7,4920,7,0.368421,2.0
3,7,4945,3,0.600000,3.0
4,7,6361,5,0.250000,NaN


In [45]:
del op
gc.collect()

54

## 2.4 Merge all features
We now merge the DataFrames with the three types of predictors that we have created (i.e., for the users, the products and the combinations of users and products).

We will start from the **uxp** DataFrame and we will add the user and prd DataFrames. We do so because we want our final DataFrame (which will be called **data**) to have the following structure: 

<img style="float: left;" src="https://i.imgur.com/mI5BbFE.jpg" >






### 2.4.1 Merge uxp with user DataFrame
Here we select to perform a left join of uxp with user DataFrame based on matching key "user_id"

<img src="https://i.imgur.com/WlI84Ud.jpg" width="400">

Left join, ensures that the new DataFrame will have:
- all the observations of the uxp (combination of user and products) DataFrame 
- all the **matching** observations of user DataFrame with uxp based on matching key **"user_id"**

The new DataFrame as we have already mentioned, will be called **data**.

In [46]:
#Merge uxp features with the user features
#Store the results on a new DataFrame
data = uxp.merge(user, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket
0,7,274,1,0.052632,NaN,20,0.669903,13.546392,10.3
1,7,519,2,0.100000,NaN,20,0.669903,13.546392,10.3
2,7,4920,7,0.368421,2.0,20,0.669903,13.546392,10.3
3,7,4945,3,0.600000,3.0,20,0.669903,13.546392,10.3
4,7,6361,5,0.250000,NaN,20,0.669903,13.546392,10.3


In [47]:
del uxp, user
gc.collect()

54

### 2.4.1 Merge data with prd DataFrame 📚📝
In this step we continue with our new DataFrame **data** and we perform a left join with prd DataFrame. The matching key here is the "product_id".
<img src="https://i.imgur.com/Iak6nIz.jpg" width="400">

Left join, ensures that the new DataFrame will have:
- all the observations of the data (features of userXproducts and users) DataFrame 
- all the **matching** observations of prd DataFrame with data based on matching key **"product_id"**

In [48]:
#Merge uxp & user features (the new DataFrame) with prd features
data = data.merge(prd, on='product_id', how='left') #
data.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket,prd_t_purchases,p_reorder_ratio,aop_mean
0,7,274,1,0.052632,NaN,20,0.669903,13.546392,10.3,232,0.301724,10.719828
1,7,519,2,0.100000,NaN,20,0.669903,13.546392,10.3,111,0.657658,8.549550
2,7,4920,7,0.368421,2.0,20,0.669903,13.546392,10.3,8126,0.666010,8.335343
3,7,4945,3,0.600000,3.0,20,0.669903,13.546392,10.3,599,0.732888,7.193656
4,7,6361,5,0.250000,NaN,20,0.669903,13.546392,10.3,333,0.756757,7.657658


In [49]:
del prd
gc.collect()

75

### 2.4.2 Delete previous DataFrames

The information from the DataFrames that we have created to store our features (op, user, prd, uxp) is now stored on **data**. 

As we won't use them anymore, we now delete them.

# 3. Create train and test DataFrames
## 3.1 Include information about the last order of each user

The **data** DataFrame that we have created on the previous chapter (2.4) should include two more columns which define the type of user (train or test) and the order_id of the future order.
This information can be found on the initial orders DataFrame which was provided by Instacart: 

<img style="float: left;" src="https://i.imgur.com/jbatzRY.jpg" >


Towards this end:
1. We select the **orders** DataFrame to keep only the future orders (labeled as "train" & "test). 
2. Keep only the columns of our desire ['eval_set', 'order_id'] <span style="color:red">**AND** </span> 'user_id' as is the matching key with our **data** DataFrame
2. Merge **data** DataFrame with the information for the future order of each customer using as matching key the 'user_id'

To filter and select the columns of our desire on orders (the 2 first steps) there are numerous approaches:

In [50]:
## First approach:
# In two steps keep only the future orders from all customers: train & test 
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

## Second approach (if you want to test it you have to re-run the notebook):
# In one step keep only the future orders from all customers: train & test 
#orders_future = orders.loc[((orders.eval_set=='train') | (orders.eval_set=='test')), ['user_id', 'eval_set', 'order_id'] ]
#orders_future.head(10)

## Third approach (if you want to test it you have to re-run the notebook):
# In one step exclude all the prior orders so to deal with the future orders from all customers
#orders_future = orders.loc[orders.eval_set!='prior', ['user_id', 'eval_set', 'order_id'] ]
#orders_future.head(10)

,user_id,eval_set,order_id
74,7,train,525192
129,14,train,2316178
272,22,test,139655
296,24,train,965160
439,29,train,3110252
485,34,train,698604
570,38,train,3173750
588,40,test,2431024
669,48,train,2924697
999,64,train,2639013


Now to fulfill step 3, we merge on **data** DataFrame the information for the last order of each customer. The matching key here is the user_id and we select a left join as we want to keep all the observations from **data** DataFrame.

<img src="https://i.imgur.com/m3pNVDW.jpg" width="400">

In [51]:
# bring the info of the future orders to data DF
data = data.merge(orders_future, on='user_id', how='left')
data.head(10)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket,prd_t_purchases,p_reorder_ratio,aop_mean,eval_set,order_id
0,7,274,1,0.052632,NaN,20,0.669903,13.546392,10.3,232,0.301724,10.719828,train,525192
1,7,519,2,0.100000,NaN,20,0.669903,13.546392,10.3,111,0.657658,8.549550,train,525192
2,7,4920,7,0.368421,2.0,20,0.669903,13.546392,10.3,8126,0.666010,8.335343,train,525192
3,7,4945,3,0.600000,3.0,20,0.669903,13.546392,10.3,599,0.732888,7.193656,train,525192
4,7,6361,5,0.250000,NaN,20,0.669903,13.546392,10.3,333,0.756757,7.657658,train,525192
5,7,8277,3,0.375000,1.0,20,0.669903,13.546392,10.3,8990,0.755617,7.466518,train,525192
6,7,8518,3,0.157895,1.0,20,0.669903,13.546392,10.3,6828,0.627124,9.239309,train,525192
7,7,9598,3,0.157895,1.0,20,0.669903,13.546392,10.3,523,0.684512,9.445507,train,525192
8,7,10504,1,0.200000,1.0,20,0.669903,13.546392,10.3,731,0.492476,9.121751,train,525192
9,7,10895,3,0.166667,NaN,20,0.669903,13.546392,10.3,547,0.594150,9.665448,train,525192


## 3.2 Prepare the train DataFrame 📚📝
In order to prepare the train Dataset, which will be used to create our prediction model, we need to include also the response (Y) and thus have the following structure:

<img style="float: left;" src="https://i.imgur.com/PDu2vfR.jpg" >

Towards this end:
1. We keep only the customers who are labelled as "train" from the competition
2. For these customers we get from order_products_train the products that they have bought, in order to create the response variable (reordered:1 or 0)
3. We make all the required manipulations on that dataset and we remove the columns that are not predictors

So now we filter the **data** DataFrame so to keep only the train users:

In [52]:
#Keep only the customers who we know what they bought in their future order
data_train = data[data.eval_set=='train'] #
data_train.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket,prd_t_purchases,p_reorder_ratio,aop_mean,eval_set,order_id
0,7,274,1,0.052632,NaN,20,0.669903,13.546392,10.3,232,0.301724,10.719828,train,525192
1,7,519,2,0.100000,NaN,20,0.669903,13.546392,10.3,111,0.657658,8.549550,train,525192
2,7,4920,7,0.368421,2.0,20,0.669903,13.546392,10.3,8126,0.666010,8.335343,train,525192
3,7,4945,3,0.600000,3.0,20,0.669903,13.546392,10.3,599,0.732888,7.193656,train,525192
4,7,6361,5,0.250000,NaN,20,0.669903,13.546392,10.3,333,0.756757,7.657658,train,525192


For these customers we get from order_products_train the products that they have bought. The matching keys are here two: the "product_id" & "order_id". A left join keeps all the observations from data_train DataFrame

<img src="https://i.imgur.com/kndys9d.jpg" width="400">

In [53]:
#Get from order_products_train all the products that the train users bought bought in their future order
data_train = data_train.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
data_train.head(15)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket,prd_t_purchases,p_reorder_ratio,aop_mean,eval_set,order_id,reordered
0,7,274,1,0.052632,NaN,20,0.669903,13.546392,10.3,232,0.301724,10.719828,train,525192,NaN
1,7,519,2,0.100000,NaN,20,0.669903,13.546392,10.3,111,0.657658,8.549550,train,525192,NaN
2,7,4920,7,0.368421,2.0,20,0.669903,13.546392,10.3,8126,0.666010,8.335343,train,525192,NaN
3,7,4945,3,0.600000,3.0,20,0.669903,13.546392,10.3,599,0.732888,7.193656,train,525192,NaN
4,7,6361,5,0.250000,NaN,20,0.669903,13.546392,10.3,333,0.756757,7.657658,train,525192,NaN
5,7,8277,3,0.375000,1.0,20,0.669903,13.546392,10.3,8990,0.755617,7.466518,train,525192,NaN
6,7,8518,3,0.157895,1.0,20,0.669903,13.546392,10.3,6828,0.627124,9.239309,train,525192,NaN
7,7,9598,3,0.157895,1.0,20,0.669903,13.546392,10.3,523,0.684512,9.445507,train,525192,NaN
8,7,10504,1,0.200000,1.0,20,0.669903,13.546392,10.3,731,0.492476,9.121751,train,525192,NaN
9,7,10895,3,0.166667,NaN,20,0.669903,13.546392,10.3,547,0.594150,9.665448,train,525192,NaN


On the last columm (reordered) you can find out our response (y). 
There are combinations of User X Product which they were reordered (1) on last order where other were not (NaN value).

Now we manipulate the data_train DataFrame, to bring it into a structure for Machine Learning (X1,X2,....,Xn, y):
- Fill NaN values with value zero (regards reordered rows without value = 1)
- Set as index the column(s) that describe uniquely each row (in our case "user_id" & "product_id")
- Remove columns which are not predictors (in our case: 'eval_set','order_id')

In [54]:
#Where the previous merge, left a NaN value on reordered column means that the customers they haven't bought the product. We change the value on them to 0.
data_train['reordered'] = data_train['reordered'].fillna(0)
data_train.head(15)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket,prd_t_purchases,p_reorder_ratio,aop_mean,eval_set,order_id,reordered
0,7,274,1,0.052632,NaN,20,0.669903,13.546392,10.3,232,0.301724,10.719828,train,525192,0.0
1,7,519,2,0.100000,NaN,20,0.669903,13.546392,10.3,111,0.657658,8.549550,train,525192,0.0
2,7,4920,7,0.368421,2.0,20,0.669903,13.546392,10.3,8126,0.666010,8.335343,train,525192,0.0
3,7,4945,3,0.600000,3.0,20,0.669903,13.546392,10.3,599,0.732888,7.193656,train,525192,0.0
4,7,6361,5,0.250000,NaN,20,0.669903,13.546392,10.3,333,0.756757,7.657658,train,525192,0.0
5,7,8277,3,0.375000,1.0,20,0.669903,13.546392,10.3,8990,0.755617,7.466518,train,525192,0.0
6,7,8518,3,0.157895,1.0,20,0.669903,13.546392,10.3,6828,0.627124,9.239309,train,525192,0.0
7,7,9598,3,0.157895,1.0,20,0.669903,13.546392,10.3,523,0.684512,9.445507,train,525192,0.0
8,7,10504,1,0.200000,1.0,20,0.669903,13.546392,10.3,731,0.492476,9.121751,train,525192,0.0
9,7,10895,3,0.166667,NaN,20,0.669903,13.546392,10.3,547,0.594150,9.665448,train,525192,0.0


In [55]:
#We set user_id and product_id as the index of the DF
data_train = data_train.set_index(['user_id', 'product_id'])
data_train.head(15)

uxp_t_bought    ...      reordered
user_id product_id                  ...               
7       274                    1    ...            0.0
        519                    2    ...            0.0
        4920                   7    ...            0.0
        4945                   3    ...            0.0
        6361                   5    ...            0.0
        8277                   3    ...            0.0
        8518                   3    ...            0.0
        9598                   3    ...            0.0
        10504                  1    ...            0.0
        10895                  3    ...            0.0
        11520                  1    ...            0.0
        12196                  2    ...            0.0
        13176                  1    ...            0.0
        13198                  8    ...            1.0
        13249                  1    ...            0.0

[15 rows x 13 columns]

In [56]:
#We remove all non-predictor variables
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train.head(15)

uxp_t_bought    ...      reordered
user_id product_id                  ...               
7       274                    1    ...            0.0
        519                    2    ...            0.0
        4920                   7    ...            0.0
        4945                   3    ...            0.0
        6361                   5    ...            0.0
        8277                   3    ...            0.0
        8518                   3    ...            0.0
        9598                   3    ...            0.0
        10504                  1    ...            0.0
        10895                  3    ...            0.0
        11520                  1    ...            0.0
        12196                  2    ...            0.0
        13176                  1    ...            0.0
        13198                  8    ...            1.0
        13249                  1    ...            0.0

[15 rows x 11 columns]

## 3.3 Prepare the test DataFrame 📚📝
The test DataFrame must have the same structure as the train DataFrame, excluding the "reordered" column (as it is the label that we want to predict).
<img style="float: left;" src="https://i.imgur.com/lLJ7wpA.jpg" >

 To achieve this we:
- Keep only the customers who are labelled as test
- Set as index the column(s) that uniquely describe each row (in our case "user_id" & "product_id")
- Remove the columns that are predictors (in our case:'eval_set', 'order_id')

In [57]:
#Keep only the future orders from customers who are labelled as test
data_test = data[data.eval_set=='test'] #
data_test.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,u_reordered_ratio,mean_days,mean_basket,prd_t_purchases,p_reorder_ratio,aop_mean,eval_set,order_id
210,22,2452,2,0.181818,NaN,15,0.507246,11.952381,4.6,1096,0.621350,10.065693,test,139655
211,22,4217,1,0.333333,1.0,15,0.507246,11.952381,4.6,68,0.558824,11.014706,test,139655
212,22,4421,1,0.090909,NaN,15,0.507246,11.952381,4.6,1238,0.546042,9.386107,test,139655
213,22,5212,1,0.125000,NaN,15,0.507246,11.952381,4.6,2379,0.741488,7.629256,test,139655
214,22,5450,1,0.125000,NaN,15,0.507246,11.952381,4.6,5120,0.680078,7.482617,test,139655


In [58]:
#We set user_id and product_id as the index of the DF
data_test = data_test.set_index(['user_id', 'product_id']) #
data_test.head()

uxp_t_bought  uxp_reorder_ratio    ...     eval_set  order_id
user_id product_id                                     ...                       
22      2452                   2           0.181818    ...         test    139655
        4217                   1           0.333333    ...         test    139655
        4421                   1           0.090909    ...         test    139655
        5212                   1           0.125000    ...         test    139655
        5450                   1           0.125000    ...         test    139655

[5 rows x 12 columns]

In [59]:
#We remove all non-predictor variables
data_test = data_test.drop(['eval_set','order_id'], axis=1)
#Check if the data_test DF, has the same number of columns as the data_train DF, excluding the response variable
data_test.head()

uxp_t_bought    ...       aop_mean
user_id product_id                  ...               
22      2452                   2    ...      10.065693
        4217                   1    ...      11.014706
        4421                   1    ...       9.386107
        5212                   1    ...       7.629256
        5450                   1    ...       7.482617

[5 rows x 10 columns]

# 4. Create predictive model (fit)
The Machine Learning model that we are going to create is based on the Logistic Regression Algorithm.

From Scikit-learn package we import the LogisticRegression estimator.

To create the predictive model we:
1. We create a DataFrame with all the predictors, named **X_train** and a Series with the response, named **y_train**
2. We initiata a Logistic regression model with a specific random_state (so we can reproduce if we want to our model).
3. Finally we train our model with the X_train and y_train data.

In [60]:
########################
#IMPORT REQUIRED PACKAGES
#######################
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split #validate algorithm
#from sklearn.metrics import f1_score, classification_report, confusion_matrix

#####################
#CREATE X_train, y_train
#####################
#X_train, y_train = data_train.drop('reordered', axis=1), data_train.reordered test_size=0.8, random_state=42
#X_train, X_val, y_train, y_val = train_test_split(data_train.drop('reordered', axis=1), data_train.reordered, test_size=0.7, random_state=42)
  #validate algorithm

#####################
# INITIATE AND TRAIN MODEL
#####################
#rfc = RandomForestClassifier(n_estimators=20, n_jobs=-1 ,random_state=42)
###model = rfc.fit(X_train, y_train) #

#####################
# SCORE MODEL
#####################
#y_val_pred = (model.predict_proba(X_val)[:,1] >= 0.21).astype(int)

#####################
#PRINT SCORES
#####################
#print("RESULTS ON VALIDATION SET\n====================")
#print("F1 Score: ",f1_score(y_val, y_val_pred, average='binary'), "\n====================")
#print("Classification Report\n ", classification_report(y_val, y_val_pred), "\n====================")
#print("Confusion Matrix\n ", confusion_matrix(y_val, y_val_pred))
#####################
## DELETE TEMPORARY OBJECTS 
#####################

#del [X_val, y_val] # remove validate algorithm objects
#gc.collect()

############################
# FEATURE IMPORTANCE - AS DF
############################
#feature_importances_df = pd.DataFrame(model.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
#print(feature_importances_df)

##################################
# FEATURE IMPORTANCE - GRAPHICAL
##################################
#feat_importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values()
#feat_importances.plot(kind='barh')

############################
# DELETE TEMPORARY OBJECTS #
############################
#del [X_train, y_train]
#gc.collect()

### TO REMOVE THE FUTURE WARNING THAT YOU SEE BELOW: Add the argument solver='lbfgs' to LogisticRegression( ) function

SyntaxError: invalid syntax (<ipython-input-5-c92f630583f1>, line 1)

In [6]:
## DISABLE WARNINGS
###########################


import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

###########################
## IMPORT REQUIRED PACKAGES
###########################
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

X_train, y_train = data_train.drop('reordered', axis=1), data_train.reordered

####################################
## SET BOOSTER'S RANGE OF PARAMETERS
# IMPORTANT NOTICE: Fine-tuning an XGBoost model may be a computational prohibitive process with a regular computer or a Kaggle kernel. 
# Be cautious what parameters you enter in paramiGrid section.
# More paremeters means that GridSearch will create and evaluate more models.
####################################    
paramGrid = {"max_depth":[5,10],
            "colsample_bytree":[0.3,0.4],
            "n_estimators":range(50,200,10)}  

########################################
## INSTANTIATE XGBClassifier()
########################################
xgbc = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', num_boost_round=10)

##############################################
## DEFINE HOW TO TRAIN THE DIFFERENT MODELS
#############################################
gridsearch = GridSearchCV(xgbc, paramGrid, cv=3, verbose=2, n_jobs=1)

################################################################
## TRAIN THE MODELS
### - with the combinations of different parameters
### - here is where GridSearch will be exeucuted
#################################################################
model = gridsearch.fit(X_train, y_train)

##################################
## OUTPUT(S)
##################################
# Print the best parameters
print("The best parameters are: /n",  gridsearch.best_params_)

# Store the model for prediction (chapter 5)
model = gridsearch.best_estimator_

# Delete X_train , y_train
del [X_train, y_train]

##################################
# FEATURE IMPORTANCE - GRAPHICAL
##################################
xgb.plot_importance(model)


ModuleNotFoundError: No module named 'xgboost'

In [62]:
model.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.3,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1,
 'eval_metric': 'logloss',
 'num_boost_round': 10}

In [63]:
del orders_future
gc.collect()

0

# 5. Apply predictive model (predict)
The model that we have created is stored in the **model** object.
At this step we predict the values for the test data and we store them in a new column in the same DataFrame.

In [64]:
'''
# Predict values for test data with our model from chapter 5 - the results are saved as a Python array
test_pred = model.predict(data_test).astype(int)
test_pred[0:20] #display the first 20 predictions of the numpy array
'''

'\n# Predict values for test data with our model from chapter 5 - the results are saved as a Python array\ntest_pred = model.predict(data_test).astype(int)\ntest_pred[0:20] #display the first 20 predictions of the numpy array\n'

In [65]:
## OR set a custom threshold (in this problem, 0.21 yields the best prediction)
test_pred = (model.predict_proba(data_test)[:,1] >= 0.21).astype(int)
test_pred[0:20] #display the first 20 predictions of the numpy array

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0])

In [66]:
#Save the prediction in a new column in the data_test DF
data_test['prediction'] = test_pred
data_test.head()

uxp_t_bought     ...      prediction
user_id product_id                   ...                
22      2452                   2     ...               0
        4217                   1     ...               0
        4421                   1     ...               0
        5212                   1     ...               0
        5450                   1     ...               0

[5 rows x 11 columns]

In [67]:
#Reset the index
final = data_test.reset_index()
#Keep only the required columns to create our submission file (Chapter 6)
final = final[['product_id', 'user_id', 'prediction']]

gc.collect()
final.head()

,product_id,user_id,prediction
0,2452,22,0
1,4217,22,0
2,4421,22,0
3,5212,22,0
4,5450,22,0


# 6. Creation of Submission File
To submit our prediction to Instacart competition we have to get for each user_id (test users) their last order_id. The final submission file should have the test order numbers and the products that we predict that are going to be bought.

To create this file we retrieve from orders DataFrame all the test orders with their matching user_id:

In [68]:
orders_test = orders.loc[orders.eval_set=='test',("user_id", "order_id") ]
orders_test.head()

,user_id,order_id
272,22,139655
588,40,2431024
1148,75,1970262
1445,94,622677
1875,129,688716


We merge it with our predictions (from chapter 5) using a left join:
<img src="https://i.imgur.com/KJubu0v.jpg" width="400">

In [69]:
final = final.merge(orders_test, on='user_id', how='left')
final.head()

,product_id,user_id,prediction,order_id
0,2452,22,0,139655
1,4217,22,0,139655
2,4421,22,0,139655
3,5212,22,0,139655
4,5450,22,0,139655


And we move on with two final manipulations:
- remove any undesired column (in our case user_id)
- set product_id column as integer (mandatory action to proceed to the next step)

In [70]:
#remove user_id column
final = final.drop('user_id', axis=1)
#convert product_id as integer
final['product_id'] = final.product_id.astype(int)

#Remove all unnecessary objects
del orders
del orders_test
gc.collect()

final.head()

,product_id,prediction,order_id
0,2452,0,139655
1,4217,0,139655
2,4421,0,139655
3,5212,0,139655
4,5450,0,139655


In this step we initiate an empty dictionary. In this dictionary we will place as index the order_id and as values all the products that the order will have. If none product will be purchased, we have explicitly to place the string "None". All this syntax follows the requirements of the competition for the submission file.

In [71]:
d = dict()
for row in final.itertuples():
    if row.prediction== 1:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in final.order_id:
    if order not in d:
        d[order] = 'None'
        
gc.collect()

#We now check how the dictionary were populated (open hidden output)
d

{139655: '13176 21903 22935 27845 32655',
 2431024: '5322 5450 5699 6975 13176 17044 22035 24799 27555 29926 34217 39928',
 1970262: '4274 4562 11520 12872 13176 21267 21903 22451 24964 27104 27105 28058 28849 29071 30489 44683 48364',
 688716: '3721 5077 13176 36929 47209 47626',
 1920100: '2748 9444 19173 21903 22582 22908 30391 34458 35221 37687 41658 46969 47209 49235',
 2088686: '1516 3583 5380 5479 9390 26128 27531 29837 30489 31553 39877 39928 40311 42265 46802 47626',
 2305779: '12341 22938 48174',
 2193705: '4149 6087 6184 8048 12341 13259 21903 29223 30827 38768 39275 40120 40742 48762',
 1185710: '9839 14715 19508 21137 21267 21775 21903 22935 24184 24852 24964 27548 27845 28204 30391 38689 39216 42828 43210 45633 49236',
 442242: '3957 7521 25718',
 1918388: '5134 14084 25931 27104 33120 39639 42265 42585',
 1990335: '21663 43598 43961',
 668032: '6347',
 476412: '5450 24852 32153',
 2151106: '4776 13176 16541 21137 28227 43352 47141',
 836275: '5460 10673 13293 16816 24852

Now we convert the dictionary to a DataFrame and prepare it to extact it into a .csv file

In [72]:
#Convert the dictionary into a DataFrame
sub = pd.DataFrame.from_dict(d, orient='index')

#Reset index
sub.reset_index(inplace=True)
#Set column names
sub.columns = ['order_id', 'products']

sub.head()

,order_id,products
0,139655,13176 21903 22935 27845 32655
1,2431024,5322 5450 5699 6975 13176 17044 22035 24799 27...
2,1970262,4274 4562 11520 12872 13176 21267 21903 22451 ...
3,688716,3721 5077 13176 36929 47209 47626
4,1920100,2748 9444 19173 21903 22582 22908 30391 34458 ...


**The submission file should have 75.000 predictions to be submitted in the competition**

In [73]:
#Check if sub file has 75000 predictions
sub.shape[0]

7501

The DataFrame can now be converted to .csv file. Pandas can export a DataFrame to a .csv file with the .to_csv( ) function.

In [74]:
sub.to_csv('sub.csv', index=False)

# 7. Get F1 Score

Before you are ready to submit your prediction to the competion, **ensure that**:
- **You have used all of the offered data and not the 10% that was defined as an optional step on section 1.2**

To submit your prediction and get the F1 score you have to:
1. Commit this notebook and wait for the results 
2. Go to view mode (where you see your notebook but you can't edit it)
3. Click on the data section from your left panel
4. Find the sub.csv (on outputs), below the section with the data from Instacart
5. Click on "Submit to competition" button

Regarding step 1:
>This step might take long. If it exceeds 20-30 minutes it would be wise to check your code again. Kaggle won't inform you during commit if the notebook has:
- syntax errors
- if it exceeds 16 GB RAM
- if it takes an algorirthms too much to train or predict

>Any new commit:
- can't take more than 9 hours
- doesn't stop if it exceeds the 16 GB RAM - you will just receive an error of unsuccesful commit after 9 hours